In [ ]:
import os
import numpy as np
import mne
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import glob
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# 1. Device selection (MPS, CUDA, or CPU)
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# 2. Paths and parameters
DATA_DIR = '/Users/kumarsatyam/Desktop/summer2025/deeplearning/final_project/chb-mit-scalp-eeg-database-1.0.0'
RECORDS_FILE = os.path.join(DATA_DIR, 'RECORDS')
RECORDS_WITH_SEIZURES_FILE = os.path.join(DATA_DIR, 'RECORDS-WITH-SEIZURES')
window_sec = 5
sfreq = 256
window_size = window_sec * sfreq
save_dir = "eeg_windows"
os.makedirs(save_dir, exist_ok=True)

# 3. Find common channels across all files (run once)
def get_common_channels(edf_files, max_files=20):
    common = None
    for i, file in enumerate(edf_files):
        if i >= max_files:  # Limit for speed
            break
        raw = mne.io.read_raw_edf(os.path.join(DATA_DIR, file), preload=False, verbose=False)
        chs = set([ch for ch in raw.ch_names if ch != '-'])
        if common is None:
            common = chs
        else:
            common = common & chs
    return sorted(list(common))

# --- Extraction: Only process missing .npz files ---
with open(RECORDS_FILE) as f:
    all_files = [line.strip() for line in f if line.strip()]
with open(RECORDS_WITH_SEIZURES_FILE) as f:
    seizure_files = set(line.strip() for line in f if line.strip())

# Find common channels (intersection)
print("Finding common channels...")
common_channels = get_common_channels(all_files, max_files=20)
print(f"Using {len(common_channels)} common channels: {common_channels}")

def extract_windows(file_path, label, window_size=1280):
    raw = mne.io.read_raw_edf(os.path.join(DATA_DIR, file_path), preload=True, verbose=False)
    # Only use channels present in both the file and the common set
    file_chs = set(raw.ch_names)
    pick_chs = [ch for ch in common_channels if ch in file_chs]
    if len(pick_chs) != len(common_channels):
        # Skip files missing any common channel
        print(f"Skipping {file_path}: missing channels {set(common_channels) - file_chs}")
        return None, None
    raw.pick(pick_chs)
    data = raw.get_data()
    data = (data - np.mean(data, axis=1, keepdims=True)) / (np.std(data, axis=1, keepdims=True) + 1e-8)
    X, y = [], []
    for start in range(0, data.shape[1] - window_size, window_size):
        X.append(data[:, start:start+window_size])
        y.append(label)
    return np.array(X), np.array(y)

# Only process EDFs that don't have a corresponding .npz
for file in tqdm(all_files, desc="Extracting and saving windows"):
    base = os.path.basename(file).replace('.edf', '')
    npz_path = os.path.join(save_dir, f"{base}.npz")
    if os.path.exists(npz_path):
        continue  # Already processed
    label = 1 if file in seizure_files else 0
    Xi, yi = extract_windows(file, label, window_size)
    if Xi is None or yi is None:
        continue  # File skipped due to missing channels
    np.savez_compressed(npz_path, X=Xi.astype(np.float16), y=yi.astype(np.int8))
print("Extraction complete.")

# 4. Prepare file lists for train/test split
all_npz = sorted(glob.glob(os.path.join(save_dir, "*.npz")))
np.random.seed(42)
np.random.shuffle(all_npz)
split = int(0.8 * len(all_npz))
train_files = all_npz[:split]
test_files = all_npz[split:]

# 5. Custom Dataset that loads windows on-the-fly
class EEGWindowDataset(Dataset):
    def __init__(self, npz_files):
        self.file_map = []
        for npzfile in npz_files:
            with np.load(npzfile) as data:
                n = data['X'].shape[0]
            self.file_map.extend([(npzfile, i) for i in range(n)])
        self.length = len(self.file_map)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        npzfile, i = self.file_map[idx]
        with np.load(npzfile) as data:
            x = data['X'][i].astype(np.float32)
            y = data['y'][i].astype(np.float32)
        return torch.tensor(x), torch.tensor(y)

# 6. DataLoader
train_ds = EEGWindowDataset(train_files)
test_ds = EEGWindowDataset(test_files)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=64, num_workers=4)

# 7. Custom LSTM implementation
import torch.nn as nn
import torch.nn.functional as F

# Get n_channels and window_size from a sample
with np.load(train_files[0]) as data:
    n_channels = data['X'].shape[1]
    window_size = data['X'].shape[2]

class CustomLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.W_ih = nn.Linear(input_size, 4 * hidden_size)
        self.W_hh = nn.Linear(hidden_size, 4 * hidden_size)

    def forward(self, x, hx):
        h, c = hx
        gates = self.W_ih(x) + self.W_hh(h)
        i, f, g, o = gates.chunk(4, dim=-1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        g = torch.tanh(g)
        o = torch.sigmoid(o)
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.cell = CustomLSTMCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        batch_size, seq_len, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_size, device=x.device)
        c = torch.zeros(batch_size, self.hidden_size, device=x.device)
        outputs = []
        for t in range(seq_len):
            h, c = self.cell(x[:, t, :], (h, c))
            outputs.append(h.unsqueeze(1))
        outputs = torch.cat(outputs, dim=1)  # (batch, seq_len, hidden_size)
        return outputs, (h.unsqueeze(0), c.unsqueeze(0))

class EEGNetLSTM(nn.Module):
    def __init__(self, n_channels, window_size):
        super().__init__()
        self.conv1 = nn.Conv1d(n_channels, 32, kernel_size=7, padding=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.lstm = CustomLSTM(input_size=64, hidden_size=32)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        # x: (batch, channels, time)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)  # (batch, 64, seq_len)
        x = x.permute(0, 2, 1)  # (batch, seq_len, features) for LSTM
        _, (h_n, _) = self.lstm(x)
        x = h_n[-1]  # Last hidden state
        x = self.dropout(x)
        x = self.fc(x)
        return self.sigmoid(x).squeeze()

model = EEGNetLSTM(n_channels, window_size).to(device)

# 8. Training loop with tqdm
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 10
train_losses, val_losses = [], []

for epoch in range(n_epochs):
    model.train()
    running_loss = 0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs} [Train]", leave=False)
    for xb, yb in train_bar:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
        train_bar.set_postfix(loss=loss.item())
    train_losses.append(running_loss / len(train_loader.dataset))

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        val_bar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{n_epochs} [Val]", leave=False)
        for xb, yb in val_bar:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * xb.size(0)
            val_bar.set_postfix(loss=loss.item())
    val_losses.append(val_loss / len(test_loader.dataset))

    print(f"Epoch {epoch+1}: Train Loss={train_losses[-1]:.4f}, Val Loss={val_losses[-1]:.4f}")

# 9. Evaluation & Visualization
model.eval()
y_true, y_pred_prob = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        out = model(xb).cpu().numpy()
        y_pred_prob.extend(out)
        y_true.extend(yb.numpy())
y_pred = (np.array(y_pred_prob) > 0.5).astype(int)
print(classification_report(y_true, y_pred))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

# Training curves
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss')
plt.show()

# 10. Visualize EEG Window (from test set)
for i in range(len(test_ds)):
    x, y = test_ds[i]
    if y == 1:
        idx = i
        break
x = x.numpy()
plt.figure(figsize=(12, 6))
for ch in range(x.shape[0]):
    plt.plot(x[ch] + ch*5, label=f'Ch{ch}' if ch < 5 else None)
plt.title(f"EEG Window (True: 1, Pred: {int(y_pred[idx])})")
plt.xlabel('Sample')
plt.ylabel('Amplitude (normalized)')
plt.show()

Using device: mps
Finding common channels...
Using 23 common channels: ['C3-P3', 'C4-P4', 'CZ-PZ', 'F3-C3', 'F4-C4', 'F7-T7', 'F8-T8', 'FP1-F3', 'FP1-F7', 'FP2-F4', 'FP2-F8', 'FT10-T8', 'FT9-FT10', 'FZ-CZ', 'P3-O1', 'P4-O2', 'P7-O1', 'P7-T7', 'P8-O2', 'T7-FT9', 'T7-P7', 'T8-P8-0', 'T8-P8-1']
Extracting and saving windows:   0%|                                                                                 | 0/686 [00:00<?, ?it/s]Skipping chb12/chb12_27.edf: missing channels {'FP2-F4', 'FP1-F3', 'F8-T8', 'FT10-T8', 'P4-O2', 'FT9-FT10', 'P7-O1', 'FZ-CZ', 'T8-P8-1', 'P8-O2', 'T7-FT9', 'F3-C3', 'FP2-F8', 'P3-O1', 'T8-P8-0', 'F4-C4', 'FP1-F7', 'P7-T7', 'T7-P7', 'CZ-PZ', 'C3-P3', 'F7-T7', 'C4-P4'}
Extracting and saving windows:  50%|██████████████████████████████████▌                                  | 344/686 [00:00<00:00, 1758.83it/s]Skipping chb12/chb12_28.edf: missing channels {'FP2-F4', 'FP1-F3', 'F8-T8', 'FT10-T8', 'P4-O2', 'FT9-FT10', 'P7-O1', 'FZ-CZ', 'T8-P8-1', 'P8-O2', 'T7-FT9', 'F3-C3', 'FP2-F8', 'P3-O1', 'T8-P8-0', 'F4-C4', 'FP1-F7', 'P7-T7', 'T7-P7', 'CZ-PZ', 'C3-P3', 'F7-T7', 'C4-P4'}
Skipping chb12/chb12_29.edf: missing channels {'FP2-F4', 'FP1-F3', 'F8-T8', 'FT10-T8', 'P4-O2', 'FT9-FT10', 'P7-O1', 'FZ-CZ', 'T8-P8-1', 'P8-O2', 'T7-FT9', 'F3-C3', 'FP2-F8', 'P3-O1', 'T8-P8-0', 'F4-C4', 'FP1-F7', 'P7-T7', 'T7-P7', 'CZ-PZ', 'C3-P3', 'F7-T7', 'C4-P4'}
Skipping chb13/chb13_04.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_05.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_06.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_07.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_08.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_09.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_10.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_11.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_12.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_13.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_14.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_15.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_16.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_18.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_24.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_30.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_36.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_37.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_38.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_39.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_40.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb13/chb13_47.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb15/chb15_01.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb16/chb16_18.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb16/chb16_19.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb17/chb17c_13.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Skipping chb18/chb18_01.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Extracting and saving windows:  76%|█████████████████████████████████████████████████████                 | 520/686 [00:03<00:01, 108.06it/s]Skipping chb19/chb19_01.edf: missing channels {'T8-P8-0', 'T8-P8-1', 'P7-T7', 'FT10-T8', 'FT9-FT10', 'T7-FT9'}
Extracting and saving windows: 100%|██████████████████████████████████████████████████████████████████████| 686/686 [00:04<00:00, 169.44it/s]
Extraction complete.
Class weights - Positive: 4.00, Negative: 1.0
Epoch 1/3:   0%|                                                                                                    | 0/8732 [00:00<?, ?it/s]Using device: mps
Using device: mps
Using device: mps
Using device: mps
Epoch 1/3:  26%|█████████████████████▊                                                                                                       Epoch 1/3:  96%|████████████████████████████████████████████████████████████████████████████                           Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                         Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                       Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                     Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                   Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                              Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                            Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████               Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                          Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████             Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                        Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████           Epoch 1/3:  97%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                      Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████         Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                    Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████       Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                  Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████     Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████   Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                              Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████ Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                            Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                          Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                        Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                      Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                    Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                               Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                  Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                             Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                           Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████              Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                         Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████            Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                       Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████          Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                     Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████        Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                   Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████      Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████    Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                               Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████  Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                             Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                           Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  98%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                         Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                       Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                     Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                   Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                              Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                            Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████               Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                          Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████             Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                        Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████           Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                      Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████         Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                    Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████       Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                  Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████     Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████   Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                              Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████ Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                            Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                          Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                        Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                      Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                    Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                               Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                  Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                             Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                           Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████              Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                                 Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████                                         Epoch 1/3:  99%|████████████████████████████████████████████████████████████████████████████            █████████▍| 8681/8732 [15:08:02<05:41,  6.69s/it]

Epoch 1/3: 100%|██████████████████████████████████████████████████████████████████████████████████████| 8732/8732 [15:16:04<00:00,  6.29s/it]
Using device: mps
Using device: mps
Using device: mps
Using device: mps
Epoch 1: Train Loss=1.7865, Train Acc=0.8075, Val Loss=1.9864, Val Acc=0.7885, LR=1.00e-03
Best model saved with Val Loss=1.9864
Epoch 2/3:   0%|                                                                                                    | 0/8732 [00:00<?, ?it/s]Using device: mps
Using device: mps
Using device: mps
Using device: mps
Epoch 2/3: 100%|██████████████████████████████████████████████████████████████████████████████████████| 8732/8732 [13:54:48<00:00,  5.74s/it]
Using device: mps
Using device: mps
Using device: mps
Using device: mps
Epoch 2: Train Loss=1.2957, Train Acc=0.8573, Val Loss=2.6306, Val Acc=0.8054, LR=1.00e-03
Epoch 3/3:   0%|                                                                                                    | 0/8732 [00:00<?, ?it/s]Using device: mps
Using device: mps
Using device: mps
Using device: mps
Epoch 3/3: 100%|██████████████████████████████████████████████████████████████████████████████████████| 8732/8732 [14:00:55<00:00,  5.78s/it]
Using device: mps
Using device: mps
Using device: mps
Using device: mps
Epoch 3: Train Loss=1.1011, Train Acc=0.8772, Val Loss=3.2329, Val Acc=0.7914, LR=1.00e-03
Using device: mps
Using device: mps
Using device: mps
Using device: mps

Classification Report:
              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88    104649
         1.0       0.30      0.18      0.23     21319

    accuracy                           0.79    125968
   macro avg       0.57      0.55      0.55    125968
weighted avg       0.75      0.79      0.77    125968

Test Accuracy: 0.7885